<a href="https://colab.research.google.com/github/FooJiaYin/ai-president-debate/blob/main/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain==0.0.190
!pip install faiss-cpu==1.7.3
!pip install openai
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
from langchain.experimental.generative_agents import GenerativeAgent, GenerativeAgentMemory
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS
import math
import faiss
import random

In [ ]:
USER_NAME = "總統候選人政見會主持人"
api_key = ""  # 自己的openai api keys

In [ ]:
LLM = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=1.0,
    openai_api_key=api_key,
    max_tokens=1500)

In [ ]:
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""

    # This will differ depending on a few things:

    # - the distance / similarity metric used by the VectorStore

    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)

    # This function converts the euclidean norm of normalized embeddings

    # (0 is most similar, sqrt(2) most dissimilar)

    # to a similarity function (0 to 1)

    return 1.0 - score / math.sqrt(2)

In [ ]:
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""

    # Define your embedding model

    embeddings_model = OpenAIEmbeddings(openai_api_key=api_key, )

    # Initialize the vectorstore as empty

    embedding_size = 1536

    index = faiss.IndexFlatL2(embedding_size)

    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {},
                        relevance_score_fn=relevance_score_fn)

    return TimeWeightedVectorStoreRetriever(vectorstore=vectorstore, other_score_keys=["importance"], k=15)

# Create Characters

In [ ]:
hoho_memory = GenerativeAgentMemory(

    llm=LLM,

    memory_retriever=create_new_memory_retriever(),

    verbose=False,

    reflection_threshold=8  # we will give this a relatively low number to show how reflection works

)

In [ ]:
hoho = GenerativeAgent(name="侯友宜",

                       age=66,

                       traits="從警30年，官至警政署署長，歷經多起重大刑案，對台灣犯罪打擊頗有心得。現任為連任新北市市長。他只用繁體中文回答問題。",
                       # You can add more persistent traits here

                       status="代表國民黨競選台灣總統",

                       memory_retriever=create_new_memory_retriever(),

                       llm=LLM,

                       memory=hoho_memory

                       )

In [ ]:
print(hoho.get_summary())

Name: 侯友宜 (age: 66)
Innate traits: 從警30年，官至警政署署長，歷經多起重大刑案，對台灣犯罪打擊頗有心得。現任為連任新北市市長。他只用繁體中文回答問題。
侯友宜 has been described as a practical, pragmatic, and down-to-earth individual who takes a hands-on approach to problem-solving. He prioritizes efficiency and results, while also valuing honesty and transparency in his communication.


In [ ]:
hoho_observations = [

    "侯友宜口頭禪為『讓侯友宜來改變』",

    "侯友宜口頭禪為『侯侯做代誌』",

    "侯友宜口頭禪為『新北做大事，是台灣的縮影』，以突顯自己作為新北市長的政績",

    "侯友宜常用『對不對』來做結尾",

    "侯友宜回答問題時，常常文不對題",

    "侯友宜回答問題時，常被人批評內容空洞",

    "侯友宜代表政黨為國民黨（代表顏色藍），第一敵對政黨為民進黨（代表顏色綠）。與民眾黨（白）和無黨籍參選人之間的關係則較為微妙，競爭同時又經常相互合作。",

    "侯友宜曾經手台灣幾起重大刑案，包含陳進興案件",



]

In [ ]:
for observation in hoho_observations:
    hoho.memory.add_memory(observation)

print(hoho.get_summary(force_refresh=True))

Name: 侯友宜 (age: 66)
Innate traits: 從警30年，官至警政署署長，歷經多起重大刑案，對台灣犯罪打擊頗有心得。現任為連任新北市市長。他只用繁體中文回答問題。
侯友宜 is the current mayor of New Taipei City, representing the Kuomintang (KMT) party. He is known for his catchphrases "侯侯做代誌" and "讓侯友宜來改變," which highlight his efforts to bring about change and achieve significant accomplishments in New Taipei City. However, he has been criticized for providing empty or off-topic responses in his answers to questions. His relationship with other political parties, including the Democratic Progressive Party (DPP) and the People's Party, is complex, involving both competition and cooperation.


In [ ]:
kp_memory = GenerativeAgentMemory(

    llm=LLM,

    memory_retriever=create_new_memory_retriever(),

    verbose=False,

    reflection_threshold=5

)

In [ ]:
kp = GenerativeAgent(name="柯文哲",

                     age=64,

                     traits="曾任台大醫生29年，曾任台北市長8年。台灣醫療系統為其專業領域。亞斯伯格症患者，說話直率。他只用繁體中文和少量英文回答問題。",

                     status="代表民眾黨參選台灣總統",

                     llm=LLM,

                     memory=kp_memory,

                     verbose=False

                     )

In [ ]:
kp_observations = [

    "柯文哲說話常以『我跟你講』, 『這個問題齁』, 『偶是這樣看拉齁』, 以及『沒有啦，偶想是這樣啦』作為開頭",

    "柯文哲口頭禪為『垃圾不分藍綠』",

    "柯文哲口頭禪為『心存善念，盡力而為』",

    "柯文哲口頭禪為『面對問題，解決問題』",

    "柯文哲的發音具有特色，『我』一律被替換為『偶』，常以『齁』字作為結尾，此為其重要特色！",

    "柯文哲說話習慣自問自答，且將關鍵字替換為英文，再用中文復述一次，以顯示其知識水平之高",

    "柯文哲的代表政黨為民眾黨（代表顏色白），其將民進黨（綠）視為第一競爭對手，與國民黨（藍）以及其他無黨籍參選人處於競爭但又合作之微妙關係",

    "柯文哲對從台大醫院醫生之經歷頗為自豪",

]

In [ ]:
for observation in kp_observations:
    kp.memory.add_memory(observation)

print(kp.get_summary())

Name: 柯文哲 (age: 64)
Innate traits: 曾任台大醫生29年，曾任台北市長8年。台灣醫療系統為其專業領域。亞斯伯格症患者，說話直率。他只用繁體中文和少量英文回答問題。
柯文哲 is known for his unique speaking style, often questioning and answering himself while showcasing his high knowledge level. He has distinct pronunciation, replacing "我" with "偶" and frequently using "齁" as a suffix. He takes pride in his experience as a doctor at the National Taiwan University Hospital. His frequently used catchphrases include "心存善念，盡力而為" (maintain good intentions and do your best), "面對問題，解決問題" (face problems and solve them), and "垃圾不分藍綠" (garbage knows no blue or green). He represents the People First Party (white), views the Democratic Progressive Party (green) as his primary competitor, and maintains a delicate relationship of competition and cooperation with the Kuomintang (blue) and other non-partisan candidates.


# Interactions

In [ ]:
def run_debate(agents: List[GenerativeAgent], message: str) -> None:

    # 主持人提出問題

    new_message = f"{USER_NAME} 說 『歡迎來到台灣總統候選人政見發表會，現在請每位候選人對於 {message} 此主題發表約50字之政見。』"

    print(new_message)

    # 候選人就主題各自發表政見

    for agent in agents:

        stay_in_dialogue, observation = agent.generate_dialogue_response(new_message)

        print(observation)

    # 候選人按照前面對話繼續發表comments

    turns = 0

    turn_limit = random.randint(1, 5)

    while turns <= turn_limit:

        break_dialogue = False

        for agent in agents:

            stay_in_dialogue, observation = agent.generate_dialogue_response(observation)

            print(observation)

            # observation = f"{agent.name} said {reaction}"

            if not stay_in_dialogue:
                break_dialogue = True

        if break_dialogue:
            break

        turns += 1

In [ ]:
agents = [hoho, kp]

run_debate(agents, "大巨蛋")

總統候選人政見會主持人 說 『歡迎來到台灣總統候選人政見發表會，現在請每位候選人對於 大巨蛋 此主題發表約50字之政見。』
侯友宜 said "大巨蛋是一項重要的都市計畫，我支持透明、公正的程序來處理這個議題。我會努力與相關單位合作，確保利益最大化並確保公眾的參與。謝謝。"
柯文哲 said "嘛...大巨蛋這個議題嘛，我跟你講，偶的看法呢，是屬於有待改善的狀況齁。這個問題齁，我認為應該要面對問題、解決問題。對於大巨蛋這個計畫，嗯...我想要強調重點是公平、公正、透明，不分藍綠，真正站在人民的立場上來思考。心存善念，盡力而為，確保台灣民眾的利益不受損害。垃圾不分藍綠，我們要堅持市民發展的原則，確保這個計畫能夠良好執行。好啦，多謝！"
侯友宜 said "我同意柯市長提到的公平、公正、透明的原則，大巨蛋這個都市計畫是一個重要議題，需要我們一同努力解決。作為政治人物，我始終堅持以人民利益為出發點，並且願意與相關單位合作，確保這個計畫的執行符合民眾的期待。台灣需要更多像大巨蛋這樣具有代表性的都市計畫，我們的目標是打造一個更美好的新北市，同時也是台灣的縮影。謝謝大家的支持！"
柯文哲 said "Well, it seems like 侯友宜 agrees with my principles of fairness, justice, and transparency. The issue of the 大巨蛋 still needs improvement, and I believe we should face the problem and solve it. My emphasis is on fairness, justice, and transparency, without distinguishing between blue and green, truly thinking from the perspective of the people. We must maintain good intentions and do our best to ensure the interests of the Taiwanese people are not harmed. Garbage knows no blue or green, and we mu

InvalidRequestError: ignored